<center><h1>Exploratory Data Analysis (EDA)</h1></center>

In [ ]:
# Data Manipulation
import pandas as pd
import numpy as np
import warnings
from aws_data_processor import DataProcessor

# Visualizations
import seaborn as sns
import missingno as mso
import matplotlib.pyplot as plt
import plotly.express as px

warnings.filterwarnings("ignore") 
pd.set_option('display.max_columns', None)

# AWS tables created specifically for NOAA CO2, CH4, N2O, and SF6 data within the DynamoDB environment.
co2_table = 'CO2DataNOAA'
ch4_table = 'CH4DataNOAA'
n2o_table = 'N2ODataNOAA'
sf6_table = 'SF6DataNOAA'

<center><h1>Carbon Dioxide (CO<sub>2</sub>)</h1></center>

## Data Wrangling

In [ ]:
# Fetch and process all CO2 data from AWS database using a custom algorithm 
processor = DataProcessor(co2_table)
fetched_co2_data = processor.fetch_all_data()

# Check for empty database or DynamoDB access, otherwise proceed and process the dataframe a structured table specified in the SQL query
if fetched_co2_data.empty:
    print("No data fetched. Check DynamoDB access or table details.")
else:
    co2_df = processor.process_data(fetched_co2_data)

'Full table data retrieval completed.'

,datetime,site,ppm,latitude,longitude,altitude,elevation,intake_height,qcflag,year,month,day,season,co2_change_rate,rolling_avg_ppm,min_ppm_year,max_ppm_year,total_site_ppm_annual,gas
0,1968-01-16 20:04:00+00:00,NWR,322.36,40.05,-105.63,3526.0,3523.0,3.0,...,1968,1,16,Winter,0.0,322.360000,322.36,322.36,82,CO2
1,1968-01-16 20:35:00+00:00,NWR,322.42,40.05,-105.63,3526.0,3523.0,3.0,...,1968,1,16,Winter,0.060000000000002274,322.390000,322.36,322.42,82,CO2
2,1968-01-16 21:00:00+00:00,NWR,322.65,40.05,-105.63,3526.0,3523.0,3.0,...,1968,1,16,Winter,0.22999999999996135,322.476667,322.36,322.65,82,CO2
3,1968-01-16 21:03:00+00:00,NWR,322.46,40.05,-105.63,3526.0,3523.0,3.0,...,1968,1,16,Winter,-0.18999999999999773,322.472500,322.36,322.65,82,CO2
4,1968-01-16 21:30:00+00:00,NWR,322.56,40.05,-105.63,3526.0,3523.0,3.0,...,1968,1,16,Winter,0.10000000000002274,322.490000,322.36,322.65,82,CO2


#### Data Type Coercion

In [8]:
co2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1217013 entries, 0 to 1217012
Data columns (total 19 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   datetime               1217013 non-null  object 
 1   site                   1217013 non-null  object 
 2   ppm                    1217013 non-null  object 
 3   latitude               1217013 non-null  object 
 4   longitude              1217013 non-null  object 
 5   altitude               1217013 non-null  object 
 6   elevation              1217013 non-null  object 
 7   intake_height          1217013 non-null  object 
 8   qcflag                 1217013 non-null  object 
 9   year                   1217013 non-null  object 
 10  month                  1217013 non-null  object 
 11  day                    1217013 non-null  object 
 12  season                 1217013 non-null  object 
 13  co2_change_rate        1217013 non-null  object 
 14  rolling_avg_ppm   

#### Instert ET from ETL phase below and continue with EDA process without the need to use AWS, now that the code for AWS configuration is known, the only data that needs to be pulled is the data straight from NOAA GML. Once the full EDA phase is complete, including any feature engineering steps, then create a full preprocessor file to prepare the data for a neural network. This of what data is needed, what data is redundant. Some data such as min a max values for the year is useful for EDA, but not for the neural network. 

In [9]:
# Convert datetime to datetime type
co2_df['datetime'] = pd.to_datetime(co2_df['datetime'])

# Convert numeric columns that are currently object types to float or int as appropriate
co2_df['ppm'] = pd.to_numeric(co2_df['ppm'], errors='coerce')
co2_df['latitude'] = pd.to_numeric(co2_df['latitude'], errors='coerce')
co2_df['longitude'] = pd.to_numeric(co2_df['longitude'], errors='coerce')
co2_df['altitude'] = pd.to_numeric(co2_df['altitude'], errors='coerce')
co2_df['elevation'] = pd.to_numeric(co2_df['elevation'], errors='coerce')
co2_df['intake_height'] = pd.to_numeric(co2_df['intake_height'], errors='coerce')
co2_df['co2_change_rate'] = pd.to_numeric(co2_df['co2_change_rate'], errors='coerce')
co2_df['min_ppm_year'] = pd.to_numeric(co2_df['min_ppm_year'], errors='coerce')
co2_df['max_ppm_year'] = pd.to_numeric(co2_df['max_ppm_year'], errors='coerce')

# Convert 'year', 'month', and 'day' to integer
co2_df['year'] = co2_df['year'].astype(int)
co2_df['month'] = co2_df['month'].astype(int)
co2_df['day'] = co2_df['day'].astype(int)

# 'site', 'qcflag', 'season', and 'gas' can remain as object (string) since they are categorical
co2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1217013 entries, 0 to 1217012
Data columns (total 19 columns):
 #   Column                 Non-Null Count    Dtype              
---  ------                 --------------    -----              
 0   datetime               1217013 non-null  datetime64[ns, UTC]
 1   site                   1217013 non-null  object             
 2   ppm                    1217013 non-null  float64            
 3   latitude               1217013 non-null  float64            
 4   longitude              1217013 non-null  float64            
 5   altitude               1217013 non-null  float64            
 6   elevation              1217013 non-null  float64            
 7   intake_height          1217013 non-null  float64            
 8   qcflag                 1217013 non-null  object             
 9   year                   1217013 non-null  int32              
 10  month                  1217013 non-null  int32              
 11  day                    1

In [30]:
# Set 'datetime' as the index to facilitate time-based grouping
co2_df.set_index('datetime', inplace=True)

In [23]:
print(co2_df.shape)
co2_df.head().style.format(precision=2).set_properties(**{'border': '1.5px solid blue'}).background_gradient(cmap='coolwarm')

(1217013, 19)


,datetime,site,ppm,latitude,longitude,altitude,elevation,intake_height,qcflag,year,month,day,season,co2_change_rate,rolling_avg_ppm,min_ppm_year,max_ppm_year,total_site_ppm_annual,gas
0,1968-01-16 20:04:00+00:00,NWR,322.36,40.05,-105.63,3526.00,3523.00,3.00,...,1968,1,16,Winter,0.00,322.36,322.36,322.36,82,CO2
1,1968-01-16 20:35:00+00:00,NWR,322.42,40.05,-105.63,3526.00,3523.00,3.00,...,1968,1,16,Winter,0.06,322.39,322.36,322.42,82,CO2
2,1968-01-16 21:00:00+00:00,NWR,322.65,40.05,-105.63,3526.00,3523.00,3.00,...,1968,1,16,Winter,0.23,322.48,322.36,322.65,82,CO2
3,1968-01-16 21:03:00+00:00,NWR,322.46,40.05,-105.63,3526.00,3523.00,3.00,...,1968,1,16,Winter,-0.19,322.47,322.36,322.65,82,CO2
4,1968-01-16 21:30:00+00:00,NWR,322.56,40.05,-105.63,3526.00,3523.00,3.00,...,1968,1,16,Winter,0.10,322.49,322.36,322.65,82,CO2


In [20]:
co2_df.describe()

,ppm,latitude,longitude,altitude,elevation,intake_height,year,month,day,co2_change_rate,rolling_avg_ppm,min_ppm_year,max_ppm_year,total_site_ppm_annual
count,1.217013e+06,1.217013e+06,1.217013e+06,1.217013e+06,1.217013e+06,1.217013e+06,1.217013e+06,1.217013e+06,1.217013e+06,1.217013e+06,1.217013e+06,1.217013e+06,1.217013e+06,1.217013e+06
mean,4.010242e+02,3.708131e+01,-9.273487e+01,7.589410e+02,6.085974e+02,1.503436e+02,2.012775e+03,6.577405e+00,1.571893e+01,3.762656e-03,4.010187e+02,3.848675e+02,4.348153e+02,6.639868e+03
std,1.861633e+01,2.177140e+01,3.902159e+01,7.864138e+02,8.378323e+02,1.559775e+02,7.604845e+00,3.451018e+00,8.795548e+00,4.169980e+00,1.812243e+01,1.979547e+01,3.174333e+01,2.889133e+03
min,3.157100e+02,-8.998000e+01,-1.773800e+02,-1.200000e+00,0.000000e+00,-3.234200e+03,1.968000e+03,1.000000e+00,1.000000e+00,-1.324620e+02,3.193918e+02,3.157100e+02,3.223600e+02,1.000000e+00
25%,3.918960e+02,3.340570e+01,-1.050040e+02,3.407000e+02,5.240000e+01,1.830000e+01,2.010000e+03,4.000000e+00,8.000000e+00,-7.270000e-01,3.923632e+02,3.725780e+02,4.192260e+02,6.470000e+03
50%,4.027610e+02,4.005000e+01,-9.135290e+01,4.860000e+02,2.510000e+02,1.070000e+02,2.014000e+03,7.000000e+00,1.600000e+01,-6.000000e-03,4.028895e+02,3.869700e+02,4.354950e+02,8.052000e+03
75%,4.136730e+02,4.503450e+01,-7.835000e+01,7.160000e+02,6.114300e+02,2.440000e+02,2.018000e+03,1.000000e+01,2.300000e+01,6.780000e-01,4.135391e+02,3.991120e+02,4.529090e+02,8.450000e+03
max,5.590190e+02,8.245080e+01,1.748710e+02,4.469000e+03,4.464000e+03,4.840000e+02,2.023000e+03,1.200000e+01,3.100000e+01,1.401300e+02,4.883543e+02,4.411040e+02,5.590190e+02,9.104000e+03


#### Check For Missing Values

In [12]:
print(co2_df.isnull().sum())

datetime                 0
site                     0
ppm                      0
latitude                 0
longitude                0
altitude                 0
elevation                0
intake_height            0
qcflag                   0
year                     0
month                    0
day                      0
season                   0
co2_change_rate          0
rolling_avg_ppm          0
min_ppm_year             0
max_ppm_year             0
total_site_ppm_annual    0
gas                      0
dtype: int64


#### Check For Duplicate Values

In [21]:
def duplicates(df):
    before = df.shape[0]
    df = df.drop_duplicates()
    after = df.shape[0]

    total_duplicates = before - after
    print(f"There are a total of {total_duplicates} duplicates in this dataset.")
duplicates(co2_df)

There are a total of 0 duplicates in this dataset.


Since 'altitude' is just the sum of 'elevation' and 'intake_height', these features can be dropped as they don't add any useful information with regards to emission analysis. Also, since the qcflag was only necessary for database ingestion, it can now be dropped for the EDA phase.

In [27]:
co2_df = co2_df.drop(columns=['elevation', 'intake_height', 'qcflag'])

In [32]:
print(co2_df.shape)
co2_df.head()

(1217013, 15)


,site,ppm,latitude,longitude,altitude,year,month,day,season,co2_change_rate,rolling_avg_ppm,min_ppm_year,max_ppm_year,total_site_ppm_annual,gas
datetime,,,,,,,,,,,,,,,
1968-01-16 20:04:00+00:00,NWR,322.36,40.05,-105.63,3526.0,1968,1,16,Winter,0.00,322.360000,322.36,322.36,82,CO2
1968-01-16 20:35:00+00:00,NWR,322.42,40.05,-105.63,3526.0,1968,1,16,Winter,0.06,322.390000,322.36,322.42,82,CO2
1968-01-16 21:00:00+00:00,NWR,322.65,40.05,-105.63,3526.0,1968,1,16,Winter,0.23,322.476667,322.36,322.65,82,CO2
1968-01-16 21:03:00+00:00,NWR,322.46,40.05,-105.63,3526.0,1968,1,16,Winter,-0.19,322.472500,322.36,322.65,82,CO2
1968-01-16 21:30:00+00:00,NWR,322.56,40.05,-105.63,3526.0,1968,1,16,Winter,0.10,322.490000,322.36,322.65,82,CO2


## Time Series Analysis

## Spatial Analysis

## Correlation Analysis